#### 1. Import Libraries and Set Environment Variables

In [3]:
!pip install faiss-cpu
!pip install transformers
!pip install python-dotenv
!pip install PyPDF2
!pip install langchain

In [4]:
import os
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Disable WANDB to avoid logging issues during training or inference
os.environ['WANDB_DISABLED'] = "true"

# Constants for file paths and model
PDF_DIRECTORY = "/path/to/directory/containing/papers/"     # Directory path containing PDF papers
PDF_FILENAME = "your_paper_name.pdf"                        # Name of the PDF file
MODEL_NAME = "/path/to/your/model/"                         # Path to the model directory


/home/lab/miniconda3/envs/halyn/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 2. Define Functions

In [5]:
def load_pdf(pdf_path):
    """
    Load and extract text from a PDF.
    
    Args:
        pdf_path (str): The path to the PDF file.

    Returns:
        str: Extracted text from the PDF.
    """
    if os.path.exists(pdf_path):
        with open(pdf_path, "rb") as f:
            pdf_reader = PdfReader(f)
            text = "".join(page.extract_text() for page in pdf_reader.pages)
        return text
    else:
        raise FileNotFoundError(f"Error: PDF file '{pdf_path}' not found.")


def split_text(text):
    """
    Split the extracted text into chunks.
    
    Args:
        text (str): The full text extracted from the PDF.

    Returns:
        list: A list of text chunks.
    """
    text_splitter = CharacterTextSplitter(
        separator="\n",             
        chunk_size=1000,            
        chunk_overlap=200,          
        length_function=len         
    )
    return text_splitter.split_text(text)


def create_knowledge_base(chunks):
    """
    Create a FAISS knowledge base from text chunks.
    
    Args:
        chunks (list): A list of text chunks.
        
    Returns:
        FAISS: A FAISS knowledge base object.
    """
    embeddings = HuggingFaceEmbeddings()
    return FAISS.from_texts(chunks, embeddings)


def load_model(model_name):
    """
    Load the HuggingFace model and tokenizer, and create a text-generation pipeline.
    
    Args:
        model_name (str): The name of the pre-trained model.

    Returns:
        pipeline: A HuggingFace pipeline for text generation.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=150, temperature=0.1)


#### 3. Main Execution Flow

In [6]:
def main():
    """ Main function to run the PDF question-answering system. """

    pdf_path = os.path.join(PDF_DIRECTORY, PDF_FILENAME)
    
    # Load and process the PDF
    try:
        text = load_pdf(pdf_path)
    except FileNotFoundError as e:
        print(e)
        return

    chunks = split_text(text)
    knowledge_base = create_knowledge_base(chunks)

    # Load the language model
    try:
        pipe = load_model(MODEL_NAME)  # Use the updated model name
    except Exception as e:
        print(f"Error loading model: {e}")
        return

    llm = HuggingFacePipeline(pipeline=pipe)
    chain = load_qa_chain(llm, chain_type="stuff")

    # Interaction loop for user questions
    while True:
        user_question = input("Ask a question about the PDF (or type 'exit' to quit): ").strip()

        if user_question.lower() == 'exit':
            print("Exiting the program.")   # Exit message
            break

        if user_question:
            # Perform similarity search and question-answering
            docs = knowledge_base.similarity_search(user_question)  
            response = chain.run(input_documents=docs, question=user_question)

            if "Helpful Answer:" in response:
                response = response.split("Helpful Answer:")[1].strip()

            print("Response:", response)

#### 4. Run the Program

In [7]:
main()

/tmp/ipykernel_1755089/2817442548.py:49: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/tmp/ipykernel_1755089/2817442548.py:49: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/home/lab/miniconda3/envs/halyn/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprec

Response: Attention is a function that maps a query and a set of key-value pairs to an output. The output is computed as a weighted sum of the values, where the weight assigned to each value is computed by a compatibility function of the query with the corresponding key.


/home/lab/miniconda3/envs/halyn/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
